In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings, Pair_Hinge_Settings, Triplet_Distance_Settings
from features.models import GCNNSmall0, GCNNMedium0, GCNNMedium1, GCNNLarge0
from learn import get_feature_model, train_features, test_features
from datasets import load_cifar10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data, samplers, pair_samplers, triplet_samplers = load_cifar10(augment=True)
d=data; s=triplet_samplers

CIFAR-10 loaded in 105 seconds


In [4]:
tf.reset_default_graph()
f, f_params = get_feature_model(feature_model=GCNNLarge0, settings=Triplet_Distance_Settings,
                                num_features=10, data_format=[32, 32, 3],
                                f_lr=0.03, f_reg=0.001, f_scope="d_gl0_cifar10_10")
fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "d_gl0_cifar10_10"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fs.restore(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")
    train_features(sess=sess, feature_model=f, samplers=s, num_steps=25000, keep_prob=0.6, f_params=f_params)
    fs.save(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")
    test_features(sess=sess, feature_model=f, samplers=s, f_params=f_params)

INFO:tensorflow:Restoring parameters from ./models/cifar10/d_gl0_cifar10_10.ckpt
begin training features
	step 0: train loss 0.179854, train error 0.05, valid loss 0.117566, valid error 0.02
end training features // time elapsed: 2.5568 s
validation set error: 0.0379 // time elapsed: 143.8503 s


In [5]:
from classes.models import LinearClassifier0, LinearClassifier1, LinearClassifier2, LinearClassifier3, TwoLayerClassifier0
from learn import get_feature_model, get_classifier, train_features, train_classifier, test_model, test_features

In [6]:
tf.reset_default_graph()
f, f_params = get_feature_model(feature_model=GCNNLarge0, settings=Triplet_Distance_Settings,
                                num_features=10, data_format=[32, 32, 3],
                                f_lr=0.07, f_reg=0.001, f_scope="d_gl0_cifar10_10")
c, c_params = get_classifier(classifier=LinearClassifier0, num_features=10, num_classes=10, 
                             c_lr=0.01, c_reg=0.001, c_scope="meh", f_params=f_params)
fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "d_gl0_cifar10_10"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fs.restore(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")
    train_classifier(sess=sess, feature_model=f, classifier=c, data=d, samplers=s, num_steps=10000, 
                     f_params=f_params, c_params=c_params)
    test_model(sess=sess, feature_model=f, classifier=c, samplers=s, data=d, f_params=f_params, c_params=c_params)

INFO:tensorflow:Restoring parameters from ./models/cifar10/d_gl0_cifar10_10.ckpt
begin training classifier
	step 0: train loss 2.5371, train error 0.895238
end training classifier // time elapsed: 1.2749 s
validation set error: 0.8906 // time elapsed: 1.0398 s
train set error: 0.8991 // time elapsed: 13.0393 s
validation set error: 0.8906 // time elapsed: 0.7550 s
test set error: 0.8968 // time elapsed: 1.4982 s
